### 전력 상태 데이터 기반 이상탐지

<br>

[전력 설비 에너지 패턴 및 고장 분석 센서](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=239)
<br>[전력 상태 데이터 기반 이상탐지](https://aifactory.space/task/2642/overview)

<br>

In [7]:
import sys
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

x_train_path = 'data/train.csv'

train = pd.read_csv(x_train_path)
train =train.dropna()

for column in train.columns:
    if train[column].dtype in [float, int]:
        # 컬럼의 최소값과 최대값 계산
        min_val = train[column].min()
        max_val = train[column].max()

        # Min-Max 스케일링 적용
        train[column] = (train[column] - min_val) / (max_val - min_val)

data = torch.FloatTensor(train.values)
dataloader = DataLoader(data, batch_size= 1024,shuffle=False)

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(35, 16),  # 인코더 입력 차원을 조절할 수 있습니다.
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 35),  # 디코더 출력 차원을 조절할 수 있습니다.
            nn.Sigmoid(),  # 0과 1 사이의 값으로 출력
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


# 학습
correct = 0
total = 0
num_epochs = 10
for epoch in range(num_epochs):
  running_loss = 0.0
  model.train()
  for batch in dataloader:
    inputs = batch

    # 경사 초기화
    optimizer.zero_grad()

    # 순전파
    outputs = model(inputs)
    loss = criterion(outputs, inputs)

    # 역전파 및 가중치 업데이트
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {running_loss/len(dataloader)}')

print('학습이 완료되었습니다.')


Epoch [1/10] Loss: 0.07085090702356295
Epoch [2/10] Loss: 0.0682684583372848
Epoch [3/10] Loss: 0.06544640313747317
Epoch [4/10] Loss: 0.06235591244212417
Epoch [5/10] Loss: 0.0590531712527885
Epoch [6/10] Loss: 0.055615015501199766
Epoch [7/10] Loss: 0.05210197318431943
Epoch [8/10] Loss: 0.048584660385237184
Epoch [9/10] Loss: 0.04515658882121707
Epoch [10/10] Loss: 0.04190996254599372
학습이 완료되었습니다.


In [8]:
model.eval()

mse_list = []
with torch.no_grad():
  output = model(data)

for i in range(len(data)):
    mse = ((data[i] - output[i])**2).mean()
    mse_list.append(mse)

In [9]:
import numpy as np
Q1 = np.percentile(mse_list, 25)
Q3 = np.percentile(mse_list, 75)
IQR = Q3 - Q1
threshold = Q3 + 1.5 * IQR

In [10]:
x_test = pd.read_csv('data/test_x.csv')
data = torch.FloatTensor(x_test.values)

mse_list = []

model.eval()  # 모델을 평가 모드로 설정
with torch.no_grad():
    for i in range(data.shape[0]):
        input_data = data[i].unsqueeze(0)  # 데이터를 모델의 입력 형식에 맞게 변환
        reconstructed_data = model(input_data)
        mse = nn.MSELoss()(reconstructed_data, input_data)
        mse_list.append(mse.item())

th = threshold

y_pred = [0 if mse < th else 1 for mse in mse_list]
y_pred_label = pd.DataFrame(y_pred, columns = ['LABEL'])

In [11]:
y_pred_save_path = 'label.csv'
y_pred_label.to_csv(y_pred_save_path, index = False)